# **Importations datasets**

In [3]:

#! === IMPORT ===
import pandas as pd

#* Les datasets importants
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')

#* Les datasets intéressants
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_12116\2476917906.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_12116\2476917906.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')


# **Initialisation Principal**

In [5]:

#! === SETUP ===
#? title_basics

#* Garder uniquement les films
df_title_basics_clean = df_title_basics[df_title_basics['titleType'] == 'movie']

#* Supprimer les lignes avec des données manquantes essentielles
df_title_basics_clean = df_title_basics_clean.dropna(subset=['startYear', 'genres'])

#* Convertir startYear en entier et filtrer les années valides
df_title_basics_clean['startYear'] = df_title_basics_clean['startYear'].astype(int)
df_title_basics_clean = df_title_basics_clean[(df_title_basics_clean['startYear'] >= 1970) & (df_title_basics_clean['startYear'] <= 2024)]

#* Garder les colonnes essentiels
df_title_basics_clean = df_title_basics_clean[['tconst', 'primaryTitle', 'startYear', 'genres']]



#! === SETUP ===
#? title_rating

#* Supprimer les films avec peu de votes
df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]



#! === SETUP ===
#? title_akas

#* Garder uniquement les titres de film français
df_title_akas_clean = df_title_akas[df_title_akas['region'] == 'FR']

#* Garder les colonnes essentiels
df_title_akas_clean = df_title_akas_clean[['titleId', 'title']]



#! === SETUP ===
#? title_crew & name_basics

#* Supprimer les lignes avec des valeurs manquantes dans 'directors'
df_title_crew_clean = df_title_crew.dropna(subset=['directors'])

#* Remplir les valeurs manquantes avec un indicateur, par exemple 'Unknown'
df_title_crew_clean = df_title_crew_clean.fillna({'directors': 'Unknown'})

#* Transformer la colonne 'directors' en une liste
df_title_crew_clean['directors'] = df_title_crew_clean['directors'].str.split(',')

#* Exploser la colonne 'directors' en lignes individuelles
df_title_crew_clean = df_title_crew_clean.explode('directors')

#* Fusionner les 'directors' avec 'name_basics' pour obtenir les 'primaryName'
title_crew_with_directors = df_title_crew_clean.merge(df_name_basics[['nconst', 'primaryName']], 
                                                     how='left', 
                                                     left_on='directors', 
                                                     right_on='nconst')

#* Regrouper les résultats par 'tconst' pour remettre les réalisateurs sous forme de liste
title_crew_with_directors = title_crew_with_directors.groupby('tconst')['primaryName'].apply(list).reset_index()



# **Renommer les colonnes**

In [30]:

#! === RENAMEUP ===

#* Renommer les colonnes de title_basics
df_title_basics_clean.rename(columns={'tconst': 'ID',
                                      'primaryTitle': 'Titre',
                                      'startYear': 'Année',
                                      'genres':  'Genres'}, inplace=True)

#* Renommer les colonnes de title_ratings
df_title_ratings_clean.rename(columns={'tconst': 'ID',
                                       'averageRating': 'Note',
                                       'numVotes': 'Votes'}, inplace=True)

#* Renommer les colonnes de title_akas
df_title_akas_clean.rename(columns={'titleId': 'ID', 
                                    'title': 'Titre Français'}, inplace=True)

#* Renommer les colonnes de title_crew & name_basics
title_crew_with_directors.rename(columns={'primaryName': 'Nom du/des Réalisateur(s)',
                                          'tconst': 'ID'}, inplace=True)


C:\Users\koke7\AppData\Local\Temp\ipykernel_36388\4212332422.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title_ratings_clean.rename(columns={'tconst': 'ID',


# **Fusion Principal**

In [ ]:

#! === MERGEUP ===

#* Merge de title_basics & title_ratings
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on="ID")

#* Merge de df_merged_v1 & title_ratings
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='ID', how='left')

#* Merge de title_basics & title_ratings
df_merged_v3 = df_merged_v2.merge(title_crew_with_directors, on='ID', how='left')



#! === CLEANUP ===

#* Supprimer les doublons
df_merged_v4 = df_merged_v3.drop_duplicates(subset='ID')

#* Supprimer les films avec moins de 15 000 votes
df_merged_v4 = df_merged_v4[df_merged_v3['Votes'] > 15000]

#* Remplacer les valeurs manquantes dans 'Titre Français' par les valeurs de 'Titre'
df_merged_v4['Titre Français'] = df_merged_v4['Titre Français'].fillna(df_merged_v4['Titre'])

C:\Users\koke7\AppData\Local\Temp\ipykernel_36388\1200725942.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_merged_v4 = df_merged_v4[df_merged_v3['Votes'] > 15000]


# **Ajout de données supplémentaires**

In [55]:

#* Ajout de la colonne popularité
def categorize_votes(votes):
    if votes < 25000:
        return "Connu"
    elif 25000 <= votes < 100000:
        return "Populaire"
    elif 100000 <= votes < 500000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_merged_v4['Popularité'] = df_merged_v4['Votes'].apply(categorize_votes)

#* Ajout de la colonne décennie
df_merged_v4['Décennie'] = (df_merged_v4['Année'].astype(int) // 10) * 10

#* Repositionner les colonnes
df_merged_v4 = df_merged_v4[['ID', 'Nom du/des Réalisateur(s)', 'Titre', 'Titre Français', 'Année', 'Décennie', 'Genres', 'Note', 'Votes', 'Popularité']]

# **Exportation**

In [56]:
import pandas as pd

df_merged_v1.to_csv("df_merged_v1.csv", index=False)

df_merged_v2.to_csv("df_merged_v2.csv", index=False)

df_merged_v3.to_csv("df_merged_v3.csv", index=False)

df_merged_v4.to_csv("df_merged_v4.csv", index=False)